In [11]:
from IPython.display import HTML
HTML(filename='profile.html')

Number of variables,17
Number of observations,516180
Missing cells,0
Missing cells (%),0.0%
Duplicate rows,32806
Duplicate rows (%),6.4%
Total size in memory,87.0 MiB
Average record size in memory,176.8 B
Categorical,15
Numeric,2
Dataset has 32806 (6.4%) duplicate rows,Duplicates


In [14]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_transformer
import optuna

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

import matplotlib.pyplot as plt
import seaborn as sns

def keep_cols(df, thresh=.1):
    for i in range(len(df.isna().sum())):
        pct_missing = df.isna().sum()[i] / df.shape[0]
        if pct_missing < thresh:
            yield df.isna().sum().index[i]


def clean_data(filename):
    df = pd.read_csv(filename)
    df.loc[df['CLASIFFICATION_FINAL'] <= 3, 'CLASIFFICATION_FINAL'] = 1
    df.loc[df['CLASIFFICATION_FINAL'] > 3, 'CLASIFFICATION_FINAL'] = 0
    df.replace(97, np.nan, inplace=True)
    df.replace(99, np.nan, inplace=True)

    df = df[list(keep_cols(df))]
    df.dropna(how='any', inplace=True)

    return df

def objective(trial):
    df = clean_data('covid_data.csv')

    X, y = df.drop('CLASIFFICATION_FINAL', axis=1), df['CLASIFFICATION_FINAL']

    test_size = trial.suggest_categorical('test_size', [0.2, 0.3, 0.4])

    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=test_size, stratify=y)
    ml_model = trial.suggest_categorical('ml_model', ['xgb', 'lgbm'])

    if ml_model == 'lgbm':
        n_estimators = trial.suggest_int('n_estimators', 50, 400)
        learning_rate = trial.suggest_categorical('learning_rate', np.arange(0.01, 0.3, 0.01))
        num_leaves = trial.suggest_int('num_leaves', 20, 1000, step=20)
        max_depth = trial.suggest_int('max_depth', 5, 200)
        max_bin = trial.suggest_int('max_bin', 200, 300)
        reg_alpha = trial.suggest_int('reg_alpha', 0, 100, step=5)
        reg_lambda = trial.suggest_int('reg_lambda', 0, 100, step=5)
        min_split_gain = trial.suggest_float('min_split_gain', 0, 15)

        model = LGBMClassifier(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            num_leaves=num_leaves,
            max_depth=max_depth,
            max_bin=max_bin,
            reg_alpha=reg_alpha,
            reg_lambda=reg_lambda,
            min_split_gain=min_split_gain
        )
    else:
        n_estimators = trial.suggest_int('n_estimators', 50, 500)
        eta = trial.suggest_categorical('eta', np.arange(0.01, 0.3, 0.01))
        max_depth = trial.suggest_int('max_depth', 2, 50, step=2)

        model = XGBClassifier(
            n_estimators=n_estimators,
            eta=eta,
            max_depth=max_depth,
            use_label_encoder=False
        )

    ct = make_column_transformer(
        (StandardScaler(), df.select_dtypes(np.float64).columns)
    )

    pipe = Pipeline([
        ('ct', ct),
        ('model', model)
    ])

    pipe.fit(X_train, y_train)

    trial.set_user_attr(key='best_model', value=model)

    y_pred = pipe.predict(X_test)

    return accuracy_score(y_test, y_pred)

def callback(study, trial):
    if study.best_trial.number == trial.number:
        study.set_user_attr(key='best_model', value=trial.user_attrs['best_model'])


In [15]:
df = clean_data('covid_data.csv')

df

,USMER,MEDICAL_UNIT,SEX,PATIENT_TYPE,DATE_DIED,PNEUMONIA,AGE,DIABETES,COPD,ASTHMA,INMSUPR,HIPERTENSION,OTHER_DISEASE,CARDIOVASCULAR,OBESITY,RENAL_CHRONIC,TOBACCO,CLASIFFICATION_FINAL
0,2,1,1,1,03/05/2020,1.0,65.0,2,2,2,2,1,2,2,2,2,2,3
1,2,1,2,1,03/06/2020,1.0,72.0,2,2,2,2,1,2,2,1,1,2,5
2,2,1,2,2,09/06/2020,2.0,55.0,1,2,2,2,2,2,2,2,2,2,3
3,2,1,1,1,12/06/2020,2.0,53.0,2,2,2,2,2,2,2,2,2,2,7
4,2,1,2,1,21/06/2020,2.0,68.0,1,2,2,2,1,2,2,2,2,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,2,13,2,1,9999-99-99,2.0,40.0,2,2,2,2,2,2,2,2,2,2,7
1048571,1,13,2,2,9999-99-99,2.0,51.0,2,2,2,2,1,2,2,2,2,2,7
1048572,2,13,2,1,9999-99-99,2.0,55.0,2,2,2,2,2,2,2,2,2,2,7
1048573,2,13,2,1,9999-99-99,2.0,28.0,2,2,2,2,2,2,2,2,2,2,7


In [16]:
df.corr()

,USMER,MEDICAL_UNIT,SEX,PATIENT_TYPE,PNEUMONIA,AGE,DIABETES,COPD,ASTHMA,INMSUPR,HIPERTENSION,OTHER_DISEASE,CARDIOVASCULAR,OBESITY,RENAL_CHRONIC,TOBACCO,CLASIFFICATION_FINAL
USMER,1.000000,0.122734,-0.002797,-0.189024,0.152041,-0.057355,0.014293,0.010409,0.010236,0.003367,0.014382,-0.015684,0.009620,0.014269,0.010993,0.010200,0.040586
MEDICAL_UNIT,0.122734,1.000000,0.000872,-0.208914,0.107511,-0.085813,0.019256,0.013889,0.012951,0.011064,0.020703,0.026609,0.014104,0.019322,0.014355,0.011894,0.084297
SEX,-0.002797,0.000872,1.000000,0.090635,-0.080976,0.029989,0.009018,0.009478,0.010462,0.008145,0.009325,0.009279,0.008326,0.009951,0.008365,0.005040,-0.057837
PATIENT_TYPE,-0.189024,-0.208914,0.090635,1.000000,-0.653546,0.321827,0.009110,0.020012,0.023419,0.024545,0.007299,0.036233,0.022532,0.020124,0.019220,0.022413,-0.192193
PNEUMONIA,0.152041,0.107511,-0.080976,-0.653546,1.000000,-0.280630,-0.009222,-0.019100,-0.021007,-0.023538,-0.007773,-0.038127,-0.020993,-0.020291,-0.018020,-0.020685,0.193542
AGE,-0.057355,-0.085813,0.029989,0.321827,-0.280630,1.000000,0.004431,0.019062,0.022664,0.022974,-0.004894,0.023304,0.019417,0.015608,0.019956,0.021230,-0.151837
DIABETES,0.014293,0.019256,0.009018,0.009110,-0.009222,0.004431,1.000000,0.830592,0.830588,0.785899,0.835934,0.640088,0.812038,0.754276,0.813823,0.773410,-0.004487
COPD,0.010409,0.013889,0.009478,0.020012,-0.019100,0.019062,0.830592,1.000000,0.911629,0.858106,0.856936,0.693186,0.882857,0.816785,0.885591,0.838471,-0.010074
ASTHMA,0.010236,0.012951,0.010462,0.023419,-0.021007,0.022664,0.830588,0.911629,1.000000,0.874446,0.865789,0.703077,0.893652,0.826355,0.896795,0.847608,-0.010955
INMSUPR,0.003367,0.011064,0.008145,0.024545,-0.023538,0.022974,0.785899,0.858106,0.874446,1.000000,0.830179,0.737980,0.859590,0.789872,0.858104,0.812365,-0.009706
